In [44]:
import asyncio
import pandas as pd
import time
import os
from copy import deepcopy

In [46]:
commands_per_file = [{
    "file": file,
    "cortes": ["./mock_main.py -v aa -c {} -t 1 -i {}".format(c, file) for c in [0, 2, 6]]
} for file in os.listdir("./files")]

In [67]:
async def spawn(cmd):
    proc = await asyncio.create_subprocess_shell(
        cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE)
    stdout, _ = await proc.communicate()
    return stdout


async def run_command(cmd, ttl):
    """
        Dado um comando retorna tempo de execucao ou "ttl"
    """
    start = time.time()
    try:
        await asyncio.wait_for(spawn(cmd), timeout=ttl)
        delta = time.time() - start
        return delta
    except asyncio.TimeoutError:
        return "ttl"


async def main():
    data = deepcopy(commands_per_file)
    for i in range(0, len(commands_per_file)):
        data[i]["cortes"] = [asyncio.create_task(
            run_command(cmd, 0.5)) for cmd in data[i]["cortes"]]

        data[i]["cortes"] = [await task for task in data[i]["cortes"]]
    return data

In [66]:
#asyncio.run(main())
data = await main()

In [65]:
df = pd.DataFrame(data)
df.explode("cortes").set_index("file")

,cortes
file,
teste_2.txt,ttl
teste_2.txt,0.283755
teste_2.txt,0.489464
teste_1.txt,ttl
teste_1.txt,ttl
teste_1.txt,0.469885
